In [1]:
path = 'REC NOT CLOSED VFinale2.xlsm'
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc
import numpy as np
import datetime as dt

In [2]:
def preprocess_data(path = '../RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed


    return wb
        

In [3]:
def time_utilisation(wb):
   # exclude nonworking records
   df = wb.loc[wb['Reporting categories']!='Nonworking']
   df = df.rename(columns={'Reporting categories':'time'})

   # merge Time Category and Phase information in one -- activity
   non_audit = df[df['Time Category'].isna()]['Assignment Name']
   audit = df[~df['Time Category'].isna()]['Time Category']
   df['activity'] = pd.concat((non_audit,audit))

   # extract year from date, calculate working hours
   df['Date'] = pd.to_datetime(df['Date'])
   df = df.assign(year_month=df['Date'].dt.strftime('%B, %Y'))
   df = df.assign(working_hours=df[['Project Hours', 'Admin Hours']].sum(axis=1))

   df = df.groupby(['Resource','year_month','time','activity']).sum().drop(columns=['Project Hours', 'Admin Hours',
      'Nonworking Hours', 'Holiday', 'Total (All Entries)',
      'Utilization (All Entries)'])

   df['monthly_hours'] = df.groupby(level = [0,1]).sum()
   df['time_category_hours'] = df.groupby(level = [0,1,2]).sum()[['working_hours']]

   df['time_category_percentage']  = (100*df['time_category_hours']/df['monthly_hours']).round(1)
   df['activity_percentage'] = (100*df['working_hours']/df['time_category_hours']).round(1)

   df = df.reset_index()

   df.replace(np.nan,None,inplace=True)

   df = df.astype({'time_category_percentage':'string','activity_percentage':'string'})
   df['time_category_percentage']+='%'
   df['activity_percentage']+='%'

   df['time_stat'] = df['time_category_hours'].astype('string') + ' hours (' + df["time_category_percentage"] + ')'
   df['activity_hours']  =  df['working_hours'].astype('string') + ' hours (' + df["activity_percentage"] + ')'

   colour_map = dict(zip(df['time'].sort_values().unique(),px.colors.carto.Safe))
   colour_map['(?)'] = 'lightgrey'

   # hovertemplate='label = %{label}<br>%{color:.data}<br>Hours worked = %{value:.2f}'
   hovertemplate = 'label = %{label}<br>Activity = %{customdata[0]}<br>Working hours = %{value}<extra></extra>'
   fig = px.treemap(df,path=[px.Constant('Month'),'year_month','Resource','time','time_stat','activity','activity_hours'],
   hover_data=['time','activity_hours'],width = 1300,height=800,
   values='working_hours',color = 'time',color_discrete_map=colour_map,maxdepth=5)

   fig.update_layout(title='Time Utilization per Month and Auditor',title_x=0.5)
   fig.update_traces(hovertemplate=hovertemplate)

   return df,fig

In [4]:
wb = preprocess_data()

In [5]:
wb['Assignment Name'].value_counts()#.str.extract(r'(IOS/AUD/\d+/\d+-)([a-zA-Z0-9_\' -]+)|')[1]

Audit of UNESCO's Environmental Management System                                                            355
Advisory Engagement on Data Governance                                                                       310
Performance Audit of Communication and Information Sector                                                    296
Audit of UNESCO's Recruitment                                                                                271
Audit of UNESCO's Cluster Office in San José (Remote)                                                        258
Memo on Cross-cutting issues for Field Offices                                                               254
Audit of UNESCO Regional Office in Yaoundé                                                                   245
Audit of UNESCO's Implementation Partnership Agreements                                                      242
Performance audit of the World Heritage Centre                                                  

In [6]:
df, fig = time_utilisation(wb)

In [7]:
names = wb['Resource'].dropna().unique()

In [8]:
def options(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [9]:
import dash_bootstrap_components as dbc

In [10]:
def drpdwn(i):
    opts = options(names)
    
    return dcc.Dropdown(options = opts, value = names,id = i,multi = True,className='dropdown')

In [11]:
df['today'] = dt.datetime.today()

In [17]:
df.columns

Index(['Resource', 'year_month', 'time', 'activity', 'working_hours',
       'monthly_hours', 'time_category_hours', 'time_category_percentage',
       'activity_percentage', 'time_stat', 'activity_hours', 'today',
       'age_month'],
      dtype='object')

In [18]:
df['age_month'] = (df['today'].dt.to_period('M').view(dtype='int64') - pd.to_datetime(df['year_month']).dt.to_period('M').view(dtype='int64'))

df['age_days'] = (df['today'].dt.to_period('D').view(dtype='int64') - pd.to_datetime(df['year_month']).dt.to_period('D').view(dtype='int64'))


In [23]:
df[['age_days','age_month']].sort_values('age_days')

,age_days,age_month
1456,32,1
1281,32,1
1280,32,1
101,32,1
576,32,1
...,...,...
1060,581,19
1059,581,19
1058,581,19
436,581,19


In [13]:
df['age_month'].sort_values()

1456     1
1281     1
1280     1
101      1
576      1
        ..
1060    19
1059    19
1058    19
436     19
664     19
Name: age_month, Length: 1457, dtype: int64

In [59]:
def slider(id = 'slider'):
    # labels = [1, 3, 6, 12, 15, 18, 24]
    labels = [i for i in range(0,max(df['age_month'])+1,3)]
    values = [1] + labels[1:]

    labels[0] = '1 month'
    labels = ['1 month'] + [str(i) + ' months' for i in labels[1:-1]] + ['all months']

    # print(dict(zip(labels,values)))

    marks = {v: {'label': l} for v, l in zip([v * 32 for v in values], labels)}
    
    # slide = dcc.Slider(30, max(df['age_days']),marks = marks)
    return dcc.Slider(30, max(df['age_days']),value =  max(df['age_days']),marks = marks,className='slider',id=id)


In [60]:
slider()

Slider(min=30, max=581, marks={32: {'label': '1 month'}, 96: {'label': '3 months'}, 192: {'label': '6 months'}, 288: {'label': '9 months'}, 384: {'label': '12 months'}, 480: {'label': '15 months'}, 576: {'label': 'all months'}}, value=581, className='slider', id='slider')

In [61]:
hovertemplate = 'label = %{label}<br>Activity = %{customdata[0]}<br>Working hours = %{value}<extra></extra>'
colour_map = dict(zip(df['time'].sort_values().unique(),px.colors.carto.Safe))
colour_map['(?)'] = 'lightgrey'

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])

server = app.server

app.layout = html.Div(children = [html.Div(
			children= [html.H1('Time Utilization per Month and Auditor',className='heading'),
					html.Div([drpdwn('names'),slider()],className='dropdown'),
					dcc.Graph(figure = fig, id='graph', className='figure')
				]
			),
            dcc.Store(id='clientside_data',data=df.to_json())
		]
	)

@app.callback(
	Output('graph','figure'),
	Output('clientside_data','data'),
	Input('names','value'),
	Input('slider','value'))
def update_figure(name,days):

	to_plot = df[(df['Resource'].isin(name)) & (df['age_days']<=days)]

	fig = px.treemap(to_plot,path=[px.Constant('Month'),'year_month','Resource','time','time_stat','activity','activity_hours'],
	hover_data=['time','activity_hours'],values='working_hours',color = 'time',color_discrete_map=colour_map,maxdepth=5,width=1300, height= 800)

	# fig.update_layout(title='Time Utilization per Month and Auditor',title_x=0.5)
	fig.update_traces(hovertemplate=hovertemplate)

	return fig, to_plot.to_json(orient='records')
	
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [3]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import json

import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

available_countries = df['country'].unique()

app.layout = html.Div([
    dcc.Graph(
        id='clientside-graph-px'
    ),
    dcc.Store(
        id='clientside-figure-store-px'
    ),
    'Indicator',
    dcc.Dropdown(
        {'pop' : 'Population', 'lifeExp': 'Life Expectancy', 'gdpPercap': 'GDP per Capita'},
        'pop',
        id='clientside-graph-indicator-px'
    ),
    'Country',
    dcc.Dropdown(available_countries, 'Canada', id='clientside-graph-country-px'),
    'Graph scale',
    dcc.RadioItems(
        ['linear', 'log'],
        'linear',
        id='clientside-graph-scale-px'
    ),
    html.Hr(),
    html.Details([
        html.Summary('Contents of figure storage'),
        dcc.Markdown(
            id='clientside-figure-json-px'
        )
    ])
])


@app.callback(
    Output('clientside-figure-store-px', 'data'),
    Input('clientside-graph-indicator-px', 'value'),
    Input('clientside-graph-country-px', 'value')
)
def update_store_data(indicator, country):
    dff = df[df['country'] == country]
    return px.scatter(dff, x='year', y=str(indicator))


app.clientside_callback(
    """
    function(figure, scale) {
        if(figure === undefined) {
            return {'data': [], 'layout': {}};
        }
        const fig = Object.assign({}, figure, {
            'layout': {
                ...figure.layout,
                'yaxis': {
                    ...figure.layout.yaxis, type: scale
                }
             }
        });
        return fig;
    }
    """,
    Output('clientside-graph-px', 'figure'),
    Input('clientside-figure-store-px', 'data'),
    Input('clientside-graph-scale-px', 'value')
)


@app.callback(
    Output('clientside-figure-json-px', 'children'),
    Input('clientside-figure-store-px', 'data')
)
def generated_px_figure_json(data):
    return '```\n'+json.dumps(data, indent=2)+'\n```'


if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/
